## Deliverable 2. Create a Customer Travel Destinations Map.

In [14]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mahebourg,MU,-20.4081,57.7000,73.69,88,75,3.44,broken clouds
1,1,Punta Arenas,CL,-53.1500,-70.9167,48.31,37,0,17.27,clear sky
2,2,Aykhal,RU,66.0000,111.5000,-49.59,100,98,6.08,overcast clouds
3,3,Busselton,AU,-33.6500,115.3333,55.09,60,100,8.81,overcast clouds
4,4,Leningradskiy,RU,69.3833,178.4167,-31.25,91,3,4.41,clear sky


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mahebourg,MU,-20.4081,57.7000,73.69,88,75,3.44,broken clouds
6,6,Namibe,AO,-15.1961,12.1522,71.40,77,53,5.46,broken clouds
9,9,East London,ZA,-33.0153,27.9116,71.85,95,100,7.87,light rain
11,11,Moron,AR,-34.6534,-58.6198,79.02,71,84,5.44,broken clouds
14,14,Port Hedland,AU,-20.3167,118.5667,87.85,70,40,11.50,scattered clouds
17,17,Viedma,AR,-40.8135,-62.9967,73.99,17,62,18.30,broken clouds
19,19,Salalah,OM,17.0151,54.0924,71.69,43,0,6.91,clear sky
21,21,Kapaa,US,22.0752,-159.3190,80.58,74,40,4.00,light rain
25,25,Arraial Do Cabo,BR,-22.9661,-42.0278,76.69,89,86,22.39,overcast clouds
28,28,Avarua,CK,-21.2078,-159.7750,73.45,94,90,4.61,heavy intensity rain


In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df['Country'].replace('', np.nan, inplace=True)
preferred_cities_df.dropna(subset=['Country'], inplace=True)

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mahebourg,MU,73.69,broken clouds,-20.4081,57.7000,
6,Namibe,AO,71.40,broken clouds,-15.1961,12.1522,
9,East London,ZA,71.85,light rain,-33.0153,27.9116,
11,Moron,AR,79.02,broken clouds,-34.6534,-58.6198,
14,Port Hedland,AU,87.85,scattered clouds,-20.3167,118.5667,
17,Viedma,AR,73.99,broken clouds,-40.8135,-62.9967,
19,Salalah,OM,71.69,clear sky,17.0151,54.0924,
21,Kapaa,US,80.58,light rain,22.0752,-159.3190,
25,Arraial Do Cabo,BR,76.69,overcast clouds,-22.9661,-42.0278,
28,Avarua,CK,73.45,heavy intensity rain,-21.2078,-159.7750,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [23]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Mahebourg,MU,73.69,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
6,Namibe,AO,71.40,broken clouds,-15.1961,12.1522,Hotel Chik Chik Namibe
9,East London,ZA,71.85,light rain,-33.0153,27.9116,Tu Casa
11,Moron,AR,79.02,broken clouds,-34.6534,-58.6198,Hotel Morón
14,Port Hedland,AU,87.85,scattered clouds,-20.3167,118.5667,The Esplanade Hotel
...,...,...,...,...,...,...,...
700,Nikki,BJ,75.13,few clouds,9.9401,3.2108,Hotel John
703,Mananjary,MG,72.86,light rain,-21.2167,48.3333,Vahiny Lodge
704,Labrador,PH,78.93,overcast clouds,16.0281,120.1442,Olana Bed & Breakfast
708,Foxton,NZ,76.73,overcast clouds,-40.4667,175.3000,Celtic Motel


In [24]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))